##### Import Libraries

In [1]:
import geopandas as gpd
import requests
import os
import json
import logging

/usr/local/Caskroom/miniconda/base/envs/rs/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.0-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


##### Create Log File

In [2]:
# Give a name to your log file
LOG_FILE = 'Script_1_tiff_images.log'

# Create log file (and set configuration to INFO)
logging.basicConfig(filename=LOG_FILE,level=logging.INFO,
                    format='%(asctime)s:%(levelname)s:%(message)s')

##### Create Directory (Folder)

In [3]:
# Name your tiff folder
TIFF_FOLDER = 'Script_1_tiff_images/'

# Create Directory/ Folder to later store tiff images (wont overwrite if folder exists)
if not os.path.exists(TIFF_FOLDER):
    os.makedirs(TIFF_FOLDER)

##### Load API Credentials

In [4]:
# State your existing API credential file name
CREDENTIALS_FILE = 'credentials.json'

# Load API Credentials
with open(CREDENTIALS_FILE, 'r') as file:
    CREDENTIALS = json.load(file) # dictionary 
    APIKEY = CREDENTIALS['x-api-key'] # dict value 

##### Indicate Preferred Date Period

In [5]:
# State the preferred date period which you are interested to get tiff images on
START_DATE = '2021-12-15'
END_DATE = '2021-12-31'

### Section 1 - Prepare API Request Data

##### Read/ load Shapefile

In [6]:
# State your existing shapefile location (directory)
SHAPEFILE_DIR = "layers/POLYGON.shp"   

# Load shapefile
FARMS_DF = gpd.read_file(SHAPEFILE_DIR) # geo df
FARMS_DF.head()

,geometry
0,"POLYGON ((84.01875 26.68606, 84.01888 26.68566..."
1,"POLYGON ((78.58184 29.08218, 78.58193 29.08213..."
2,"POLYGON ((71.58564 25.78761, 71.58641 25.78688..."
3,"POLYGON ((78.93822 27.29710, 78.93849 27.29688..."
4,"POLYGON ((78.77363 24.02042, 78.77419 24.02104..."


##### F1 - Update Dataframe

In [7]:
def update_dataframe(farms_df):
    """ Add tracker variables to dataframe.

    Create an additional 'farm_index' column with farm indexes to the dataframe.
    
    Params:
        (geodataframe): shapefile dataframe

    Returns:
        (geodataframe): An updated dataframe with an additional column.
    """
    # Create an empty list
    farm_id = []   

    for i in farms_df.index:
        
        # Name farm by index numbering
        farm_ind = "farm_{0}".format(i + 1) # str
        
        # Append farm index into the empty list
        farm_id.append(farm_ind)  

    # Create a new column named 'farm_index' & filled with 'farm_id' list      
    farms_df['farm_index'] = farm_id 
    
    # Name the updated farms df
    updated_farms_df = farms_df 
    
    return updated_farms_df # Return the NEW/UPDATED dataframe

In [8]:
# Call Function 1
UPDATED_FARMS_DF = update_dataframe(FARMS_DF)
UPDATED_FARMS_DF.head()

,geometry,farm_index
0,"POLYGON ((84.01875 26.68606, 84.01888 26.68566...",farm_1
1,"POLYGON ((78.58184 29.08218, 78.58193 29.08213...",farm_2
2,"POLYGON ((71.58564 25.78761, 71.58641 25.78688...",farm_3
3,"POLYGON ((78.93822 27.29710, 78.93849 27.29688...",farm_4
4,"POLYGON ((78.77363 24.02042, 78.77419 24.02104...",farm_5


##### Write updated df into geojson file (stored within tiff image folder)

In [9]:
# Name your geojson file
GEOJSON_FILE = 'farms_df_updated.geojson'

# Write the updated farms df into a geojson file and store it within tiff image folder
  # Note: will overwrite if same geojson file exists
UPDATED_FARMS_DF.to_file(TIFF_FOLDER +'/'+ GEOJSON_FILE, driver='GeoJSON')

In [11]:
# # Read geojson file
# gpd.read_file(TIFF_FOLDER +'/'+ GEOJSON_FILE, driver='GeoJSON')  

### Section 2 - Create Functions to Make Connection with GEOS API

##### F2: Get A Farm Boundary Coordinates 

In [12]:
def farm_bound_coords(polygon):
    """ Get a farm's boundary coordinates and write them into a desired list format.
    
    Convert a farm's boundary coordinates (shapely polygon) to a nested list of 
    coordinate pairs.

    Params:
        (polygon): shapely.geometry.polygon.Polygon

    Returns:
        (list): A list of a farm's boundary coordinates.
    """
    lon, lat = polygon.exterior.coords.xy

    lon_list = list(lon)
    lat_list = list(lat)

    farm_bound_coord = [[[x, y] for x, y in zip(lon_list, lat_list)]] # List Comprehension

    return farm_bound_coord # List

In [14]:
# # Test Function 2 with 1st row of polygon from df.
# FARM_BOUND_COORDS = farm_bound_coords(UPDATED_FARMS_DF.loc[0, 'geometry'])
# FARM_BOUND_COORDS

##### F3: Make First API Call to Get Available Dates on A Farm

In [15]:
def get_avail_dates(farm_bound_coords):
    """ First API call to GEOS API to get the available dates on a farm.
    
    Using 'post' method and an API key to run an API request to sentinel/wfs-timeline 
    endpoint. 
    
    Within the API request body, the farm boundary coordinates would be indicated and a 
    preferred date period (i.e. start date and end date) would be specified so as to 
    acquire a list of available dates of that farm (polygon) during that period.

    Params:
        (list): A list containing a farm's boundary coordinates.

    Returns:
        (list): A list of dates on a farm (polygon).
    """
    
    # GEOS API endpoint to request for dates
    S_wfs_timeline_endpoint = 'https://abcd.xyz.aabb.xxyy/sentinel/wfs-timeline'

    # API Key
    api_key_header = {
        'X-API-KEY': APIKEY,
        'Accept': 'image/tiff'
    }

    # Default API Request Body (JSON format)
    S_wfs_timeline_request_body = {
        "feature": {
            "geometry": {
                "coordinates": farm_bound_coords,
                "type": "Polygon"
            },
            "properties": {},
            "type": "Feature"

        },
        "startDate": START_DATE,
        "endDate": END_DATE,
        "missionName": "S2.TILE"
    }
 
    # Send API request to endpoint using Post Method
    response = requests.post(
        url=S_wfs_timeline_endpoint,
        headers=api_key_header,
        json=S_wfs_timeline_request_body
    )
    
    # If response status code is not 200, raise an exception
    if response.status_code != 200:
        raise Exception('Unable to proceed. API Status Code:{0}'.format(response.status_code))    
        
    # Otherwise, continue. Convert API response body into text (aka str)
    response_body = response.text
    
    # Convert str to Py Dict/ JSON object 
    dict_data = json.loads(response_body)
    
    # Create an empty list
    avail_dates =[]
    
    # Loop the list (which is a dictionary of timestamps)
    for i in dict_data['dates']:
        
        # Create a list of dates (str elements)
        avail_dates.append(i['timestamp'])

    return avail_dates # List of str dates for a polygon (farm)

In [17]:
# # Test Function 3 with chosen start and end dates @ beginning of script
# AVAIL_DATES = get_avail_dates(FARM_BOUND_COORDS)
# AVAIL_DATES

##### F4: Make Second API Call to Get Tiff Image on A Farm

In [18]:
def get_tiff_image(farm_bound_coords, avail_date, indices):
    """ Second API call to GEOS API to get a farm's tiff image on a particular date.
    
    Using 'post' method and an API key, run an API request to sentinel-analytics/indices 
    endpoint.
    
    Within the API request body, input a farm's boundary coordinates and its available date
    and indicate one's choice of indices i.e. INDICES_1 or INDICES_2.
    
    INDICES_1 will provide S1_NR data while INDICES_2 will give NDVI data. Both indices are 
    in default list format. (See next 2 cells below)
    
    Params:
        (list): A list of a farm's boundary coordinates.
        (str): Available date in the format of "YYYY-MM-DD".
        (list): Choice of indices which is given in a default list format.

    Returns:
        (response): API reponse from GEOS API. Response 200 is preferred.
        
    """
    # GEOS API endpoint to request for tiff image
    SAI_endpoint = 'https://abcd.xyz.aabb.xxyy/sentinel-analytics/indices'

    # API Key
    api_key_header = {
        'X-API-KEY': APIKEY,
        'Accept': 'image/tiff'
    }

    # Default API Request Body (JSON format)
    SAI_request_body = {
        "feature": {
            "geometry": {
                "coordinates": farm_bound_coords,
                "type": "Polygon"
            },
            "properties": {},
            "type": "Feature"
        },
        "crop": False,
        "nodata": 0,
        "resolution": "10m",
        "date": avail_date,
        "layer": "S2_L2A",
        "includeTileProperties": True,
        "scaleConfig": {
            "factor": 1,
            "resampling": "nearest"
        },
        "indices": indices
    }

    # Send API request to endpoint using Post Method
    response = requests.post(url=SAI_endpoint,
                             headers=api_key_header,
                             json=SAI_request_body) 
        
    return response # type -> requests.models.Response

Two Choice of Indices

In [19]:
# 2 Choices of Indices.
  
# INDICES_1 refers to N Sensor data
INDICES_1 = [
    {
        "name": "S1_NR",
        "parameters": {
            "m": 4
        }
    },
    {
        "name": "SCL_CLP"
    },
    {
        "name": "SCL_CLM",
        "parameters": {
            "cloudyThreshold": 0.3
        }
    }
]

# INDICES_2 refers to NDVI data
INDICES_2 = [
    {
        "name": "NDVI"
    },
    {
        "name": "S1_MSI",
        "parameters": {
            "scale": 1
        }
    }
]

In [21]:
# # Test Function 4 with an available date of the farm and choice of indices
# RESPONSE = get_tiff_image(FARM_BOUND_COORDS, "2021-12-31", INDICES_1)
# RESPONSE # Output Response 200 is good!

### Section 3 - Activate GEOS API Soil Images Acquisition & Storage

##### F5:  Make API Requests to acquire tiff images of all farms

In [22]:
def get_files(indices): 
    """ Get tiff images for every farm in the updated farms dateframe and save into folder.
    
    With one's choice of indices, this function loops through the updated farm dataframe 
    to make API requests to GEOS API through activating the functions defined earlier. 
    
    A successful API response will return a tiff image of a farm on a particular available
    date. Multiple API requests would be made in this function and results in multiple tiff
    images retrieved. 
    
    These tiff images would then be stored in the folder (directory) created earlier.

    Params:
        (list): Choice of indices i.e. INDICES_1 or INDICES_2

    Returns:
        None.
    """
    # Access the first dictionary key's value of the first item in a list
    # [{"":"","":""}, {"":"","":""}, ...]
    ind_name = indices[0]['name']
    
    # Iterate every row of polygon (farm) in the dataframe
    for i in UPDATED_FARMS_DF.index:
        
        poly = UPDATED_FARMS_DF.loc[i, 'geometry'] # polygon.polygon datatype

        # Call F2 to get a farm's boundary coordinates   
        coords_list = farm_bound_coords(poly) 
        
        # Call F3 to get a list of available dates on the farm
        avail_dates = get_avail_dates(coords_list)
        
        # Iterate over each date in the list of available dates to get tiff image of farm
        for date in avail_dates:
            
            # Call F4 to get tiff image for a farm on a particular date with chosen indice.  
            response = get_tiff_image(coords_list, date, indices)  

            if response.status_code == 200:
                
                # Get content of API reponse 
                tiff_image = response.content
                
                # Farm's name by farm index
                farm_ind = UPDATED_FARMS_DF.loc[i, 'farm_index'] 
                
                # Directory to store tiff images
                path = (TIFF_FOLDER + farm_ind + '_' 
                        + date + '_' + ind_name + '.tiff')
                
                # Create a file for every tiff image retrieved from API
                with open(path, 'wb') as file:  
                    
                    # Write API response's content into the file
                    file.write(tiff_image) 
                
                logging.info('Request successful for farm {0} {1} {2}.'.format(i+1,
                                                                               date,
                                                                               ind_name))
                
            else:
                logging.info('Request failed for farm {0} {1} {2}.'.format(i+1,
                                                                           date,
                                                                           ind_name))      
    return            

In [23]:
 # Call Function 5 with choice of INDICES_1 (S1_NR)
get_files(INDICES_1)

In [24]:
 # Call Function 5 with choice of INDICES_2 (NDVI)
get_files(INDICES_2)